In [3]:
import pandas as pd
import re

# === 1) Baca semua file ===
batik_mahkota = pd.read_excel("hasil_scrape_batikmahkota.xlsx")
batik_kula = pd.read_excel("hasil_scrape_batikula.xlsx")
batik_kultur = pd.read_excel("hasil_scrape_batikkultur.xlsx")
batik_kerisOnline = pd.read_excel("hasil_scrape_batikkerisonline.xlsx")
batik_danarHadi = pd.read_excel("hasil_scrape_batikdanarhadi_id.xlsx")

# === 2) Gabungkan jadi satu DataFrame ===
df = pd.concat([batik_mahkota, batik_kula, batik_kultur, batik_kerisOnline, batik_danarHadi], ignore_index=True)

# === 3) Pisahkan judul & hashtag ===
def split_caption(caption):
    if pd.isna(caption):
        return '', ''
    hashtags = re.findall(r"#\w+", caption)
    hashtag_part = ' '.join(hashtags)
    title_part = re.sub(r"#\w+", '', caption).strip()
    return title_part, hashtag_part

df[['judul', 'hashtag']] = df['caption'].apply(lambda x: pd.Series(split_caption(x)))

# === 4) Reels: jika dominant_color kosong, tulis 'unknown' ===
df.loc[(df['media_type'] == 'reels') & (df['dominant_color'].isna()), 'dominant_color'] = 'unknown'

# === 5) Pisahkan post & reels, gabungkan post di atas reels ===
df_post = df[df['media_type'] == 'post']
df_reels = df[df['media_type'] == 'reels']
df_cleaned = pd.concat([df_post, df_reels], ignore_index=True)

# === 6) Simpan ke file Excel ===
df_cleaned.to_excel("data_cleaned_batik_tekstil.xlsx", index=False)
